In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join("..", "..", "..")))

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pysgmcmc.diagnostics.objective_functions import sinc
from pysgmcmc.models.bayesian_neural_network import BayesianNeuralNetwork
from pysgmcmc.sampling import Sampler
from pysgmcmc.stepsize_schedules import ConstantStepsizeSchedule


## Set up data ##
rng, n_datapoints = np.random.RandomState(np.random.randint(0, 10000)), 100
X_train = np.array([rng.uniform(0., 1., 1) for _ in range(n_datapoints)])
y_train = sinc(X_train)

X_test = np.linspace(0, 1, 100)[:, None]
y_test = sinc(X_test)

g = tf.Graph()

session = tf.InteractiveSession(graph=g)
with g.as_default():
    model = BayesianNeuralNetwork(
        session=session, batch_size=20, sampling_method=Sampler.RelativisticSGHMC,
        burn_in_steps=3000, n_iters=50000, 
        normalize_input=True, normalize_output=True,
        stepsize_schedule=ConstantStepsizeSchedule(np.sqrt(1e-4)),      
    )
    model.train(X_train, y_train)
    prediction_mean, prediction_variance = model.predict(X_test)

prediction_std = np.sqrt(prediction_variance)

plt.grid()

plt.plot(X_test[:, 0], y_test, label="true", color="black")
plt.plot(X_train[:, 0], y_train, "ro")

plt.plot(X_test[:, 0], prediction_mean, label="SGHMC", color="blue")
plt.fill_between(X_test[:, 0], prediction_mean + prediction_std, prediction_mean - prediction_std, alpha=0.2, color="blue")

plt.legend()
plt.show()

print()